# Métricas UAJ - Proyecto Final

Este notebook guiará el análisis de telemetría paso a paso.

**Estructura de carpetas (relativa a este notebook):**
```
analysis/
├─ data/  ← aquí van los JSON de telemetría
└─ métricas_uaj_notebook_p3.ipynb  ← este notebook
```

## Paso 1: Cargar y explorar el JSON

In [26]:
# Paso 1: Cargar y explorar todos los JSON de telemetría

import os
import glob
import json
import pandas as pd

data_dir = './data'
# 1) Listar todos los JSON
json_files = glob.glob(os.path.join(data_dir, '*.json'))
print(f"Detectados {len(json_files)} ficheros JSON en {data_dir}")

# 2) Leerlos y concatenar en un único DataFrame
dfs = []
for fp in json_files:
    with open(fp, 'r', encoding='utf-8') as f:
        data = json.load(f)
    dfs.append(pd.DataFrame(data))

df = pd.concat(dfs, ignore_index=True)
print("Total de eventos:", len(df))
print("\nTipos de evento y recuentos:\n", df['eventType'].value_counts())
df.head()


Detectados 4 ficheros JSON en ./data
Total de eventos: 16064

Tipos de evento y recuentos:
 eventType
Position      16048
GameStart         4
LevelStart        4
LevelEnd          4
GameEnd           4
Name: count, dtype: int64


,eventType,gameID,playerID,sessionID,timestamp,levelStarted,isStuck,posX,posY,posZ
0,GameStart,Damn,200468402800683,2004684028006831747132175772,1747132175772,NaN,NaN,NaN,NaN,NaN
1,LevelStart,Damn,200468402800683,2004684028006831747132175772,1747132175772,0.0,NaN,NaN,NaN,NaN
2,Position,Damn,200468402800683,2004684028006831747132175772,1747132175772,NaN,False,0.813311,5.131177,5.16052
3,Position,Damn,200468402800683,2004684028006831747132175772,1747132175772,NaN,False,0.813311,4.086732,5.16052
4,Position,Damn,200468402800683,2004684028006831747132175772,1747132175772,NaN,False,0.813311,4.002932,5.16052


## normalización


In [27]:
# Paso 2 (mejorado): normalización de timestamp con zona horaria
import pandas as pd

# 1) Convierte todo a datetime UTC y luego a CEST
df['datetime_utc'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
df['datetime']     = df['datetime_utc'].dt.tz_convert('Europe/Madrid').dt.tz_localize(None)

# 2) Mapear sessionID a pequeño índice
df['session_idx'] = df['sessionID'].astype('category').cat.codes + 1

# 3) Para cada sesión, calcula inicio y fin
session_times = (
    df.groupby('session_idx')['timestamp']
      .agg(start_ms='min', end_ms='max')
      .reset_index()
)
session_times['start_dt'] = pd.to_datetime(session_times['start_ms'], unit='ms', utc=True)    \
                                  .dt.tz_convert('Europe/Madrid').dt.tz_localize(None)
session_times['end_dt']   = pd.to_datetime(session_times['end_ms'],   unit='ms', utc=True)    \
                                  .dt.tz_convert('Europe/Madrid').dt.tz_localize(None)

# 4) Muestra el resultado
print(session_times[['session_idx','start_dt','end_dt']])


   session_idx                start_dt                  end_dt
0            1 2025-05-13 12:29:35.772 2025-05-13 12:29:35.772
1            2 2025-05-13 13:29:21.062 2025-05-13 13:29:21.062
2            3 2025-05-13 19:24:37.091 2025-05-13 19:24:37.091
3            4 2025-05-13 19:52:27.229 2025-05-13 19:52:27.229


## stucks


In [28]:
# Paso 3: Métricas de “atascos” en eventos Position

# 1) Filtrar sólo posiciones
df_pos = df[df['eventType'] == 'Position'].copy()

# 2) Recuento total y de atascos
total_positions = len(df_pos)
stuck_count    = df_pos['isStuck'].sum()          # True cuenta como 1
stuck_ratio    = stuck_count / total_positions

# 3) Mostrar resultados
print(f"Total de posiciones: {total_positions}")
print(f"Eventos stuck    : {stuck_count}")
print(f"Proporción stuck : {stuck_ratio:.2%}")


Total de posiciones: 16048
Eventos stuck    : 12
Proporción stuck : 0.07%


## analisis atascos


In [29]:
# Paso 4: Hotspots espaciales

import numpy as np

# 1) Definir tamaño de celda
cell_size = 1.0

# 2) Asignar celda a cada posición
df_pos['grid_x'] = np.floor(df_pos['posX'] / cell_size).astype(int)
df_pos['grid_z'] = np.floor(df_pos['posZ'] / cell_size).astype(int)

# 3) Contar posiciones y atascos por celda
grid_counts = (
    df_pos
    .groupby(['grid_x','grid_z'])
    .size()
    .reset_index(name='pos_count')
)
grid_stuck = (
    df_pos[df_pos['isStuck']]
    .groupby(['grid_x','grid_z'])
    .size()
    .reset_index(name='stuck_count')
)

# 4) Unir y rellenar ceros donde no haya atascos
grid = (
    grid_counts
    .merge(grid_stuck, on=['grid_x','grid_z'], how='left')
    .fillna(0)
)

# 5) Mostrar top 10 celdas con más atascos
top_hotspots = grid.sort_values('stuck_count', ascending=False).head(10)
print(top_hotspots)


     grid_x  grid_z  pos_count  stuck_count
453       0       5        107         12.0
1       -32       1        449          0.0
2       -32       2          8          0.0
3       -31      -1         42          0.0
4       -31       0         25          0.0
5       -31       1          2          0.0
6       -31       2         26          0.0
7       -31       3         26          0.0
8       -31       4          1          0.0
9       -30      -2         81          0.0


## duracion atascos


In [30]:
# Paso 5: Duración de atascos

# 1) Asegurarnos de que esté ordenado
df_pos = df_pos.sort_values('timestamp')

# 2) Crear identificador de cada secuencia stuck
mask = df_pos['isStuck']
# Cada vez que empieza un stuck (True tras un False) incrementamos el contador
df_pos['stuck_group'] = (mask & (~mask.shift(fill_value=False))).cumsum()

# 3) Agregar sólo los eventos stuck por grupo
stuck_groups = (
    df_pos[df_pos['isStuck']]
    .groupby('stuck_group')
    .agg(
        start_time_ms=('timestamp', 'min'),
        end_time_ms  =('timestamp', 'max'),
        pos_count    =('timestamp','size')        # cuántos registros tuvo ese atasco
    )
    .reset_index(drop=True)
)

# 4) Duración en segundos y estadísticas
stuck_groups['duration_s'] = (stuck_groups['end_time_ms'] - stuck_groups['start_time_ms']) / 1000

print("Resumen de duraciones de atascos (s):")
print(stuck_groups['duration_s'].describe())
print("\nTop 10 atascos más largos:")
print(stuck_groups.sort_values('duration_s', ascending=False).head(10))


Resumen de duraciones de atascos (s):
count    12.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: duration_s, dtype: float64

Top 10 atascos más largos:
   start_time_ms    end_time_ms  pos_count  duration_s
0  1747132175772  1747132175772          1         0.0
1  1747132175772  1747132175772          1         0.0
2  1747132175772  1747132175772          1         0.0
3  1747132175772  1747132175772          1         0.0
4  1747132175772  1747132175772          1         0.0
5  1747157077091  1747157077091          1         0.0
6  1747158747229  1747158747229          1         0.0
7  1747158747229  1747158747229          1         0.0
8  1747158747229  1747158747229          1         0.0
9  1747158747229  1747158747229          1         0.0
